# Get ArticDEM + SlideRule Returns

Difference SlideRule Elevations from ArticDEM

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import boto3
from sliderule import sliderule, icesat2

In [2]:
aoi_url = 'https://raw.githubusercontent.com/ICESAT-2HackWeek/dzdt/main/sites/utqiagvik-polygon.geojson'
gfa = gpd.read_file(aoi_url)
gfa.explore(style_kwds=dict(fill=False, color='magenta'))

## SlideRule Setup

In [3]:
icesat2.init("slideruleearth.io", verbose=False)

In [4]:
# Start with all defaults, just Polygon
parms = {
    "poly": sliderule.toregion(gfa)["poly"],
    # "srt": icesat2.SRT_LAND,
    # "cnf": icesat2.CNF_SURFACE_HIGH,
    # "len": 40.0,
    # "res": 20.0,
    # "maxi": 6
    "t0": "2023-01-01T00:00:00Z",
    "t1": "2024-01-01T00:00:00Z",
}


In [5]:
# Add Raster Sampling
# https://www.slideruleearth.io/web/rtd/user_guide/GeoRaster.html
parms["samples"] = {
            # "hls": {"asset": "landsat-hls", "bands": ["NDSI"]},
            "mosaic": {"asset": "arcticdem-mosaic", "algorithm":"NearestNeighbour"},
            #"mosaic": {"asset": "arcticdem-mosaic", "radius": 10.0, "zonal_stats": True},
            #"strips": {"asset": "arcticdem-strips", "algorithm": "CubicSpline"}
        }

In [6]:
# NOTE: don't do this for raster samples

# # Output to S3
# S3_OUTPUT = 's3://nasa-cryo-scratch/dzdt/utiaqvik.parquet'

# # Get CryoCloud AWS Credentials for SlideRule
# client = boto3.client('sts')

# with open(os.environ['AWS_WEB_IDENTITY_TOKEN_FILE']) as f:
#     TOKEN = f.read()

# response = client.assume_role_with_web_identity(
#     RoleArn=os.environ['AWS_ROLE_ARN'],
#     RoleSessionName=os.environ['JUPYTERHUB_CLIENT_ID'],
#     WebIdentityToken=TOKEN,
#     DurationSeconds=3600
# )


# # Update SlideRule Parameters
# parms["output"] = {
#     "path": S3_OUTPUT, 
#     "format": "parquet", 
#     "open_on_complete": False,
#     "region": "us-west-2",
#     "credentials": {
#          "aws_access_key_id": response['Credentials']['AccessKeyId'],
#          "aws_secret_access_key": response['Credentials']['SecretAccessKey'],
#          "aws_session_token": response['Credentials']['SessionToken']
#      }
# }

## Run SlideRule

In [7]:
# Check your parameters before submitting
parms

{'poly': [{'lon': -156.6430455278934, 'lat': 71.11303515926326},
  {'lon': -156.26446195120343, 'lat': 71.27727860723829},
  {'lon': -156.7080728245955, 'lat': 71.33780162227296},
  {'lon': -156.98688849401648, 'lat': 71.21627954209416},
  {'lon': -156.6430455278934, 'lat': 71.11303515926326}],
 't0': '2023-01-01T00:00:00Z',
 't1': '2024-01-01T00:00:00Z',
 'samples': {'mosaic': {'asset': 'arcticdem-mosaic',
   'algorithm': 'NearestNeighbour'}}}

In [8]:
%%time

# Run SlideRule

gf = icesat2.atl06p(parms, version='006')

print("Start:", gf.index.min().strftime('%Y-%m-%d'))
print("End:", gf.index.max().strftime('%Y-%m-%d'))
print("Reference Ground Tracks: {}".format(gf["rgt"].unique()))
print("Cycles: {}".format(gf["cycle"].unique()))
print("Elevation Measurements: {} ".format(gf.shape[0]))
gf.head(2)

Start: 2023-01-09
End: 2023-04-15
Reference Ground Tracks: [ 312  373  823 1257 1265 1326  381]
Cycles: [18 19]
Elevation Measurements: 16591 
CPU times: user 1.13 s, sys: 37.9 ms, total: 1.17 s
Wall time: 59 s


,rms_misfit,gt,dh_fit_dy,n_fit_photons,distance,w_surface_window_final,pflags,dh_fit_dx,cycle,h_mean,h_sigma,spot,segment_id,rgt,geometry,mosaic.time,mosaic.file_id,mosaic.value,mosaic.flags
time,,,,,,,,,,,,,,,,,,,
2023-01-09 23:39:51.181383424,0.134553,20,0.0,10,7.947966e+06,17.779561,0,-0.004885,18,1.625571,0.050964,5,396540,312,POINT (-156.36948 71.24078),NaN,NaN,NaN,NaN
2023-01-09 23:39:51.184223232,0.000000,20,0.0,13,7.947986e+06,54.940145,2,-2.154382,18,-28.113643,0.000000,5,396541,312,POINT (-156.36956 71.24096),NaN,NaN,NaN,NaN


In [9]:
output_file = 'utqiagvik.gpkg'
gf.to_file(output_file)

## Analyze SlideRule Results

In [10]:
output_file = 'utqiagvik.gpkg'
#gf = gpd.read_parquet(s3path)
gf = gpd.read_file(output_file)

In [11]:
len(gf)

16591

In [12]:
gf.head()

,time,rms_misfit,gt,dh_fit_dy,n_fit_photons,distance,w_surface_window_final,pflags,dh_fit_dx,cycle,h_mean,h_sigma,spot,segment_id,rgt,mosaic.time,mosaic.file_id,mosaic.value,mosaic.flags,geometry
0,2023-01-09 23:39:51.181,0.134553,20,0.0,10,7.947966e+06,17.779561,0,-0.004885,18,1.625571,0.050964,5,396540,312,NaN,NaN,NaN,NaN,POINT (-156.36948 71.24078)
1,2023-01-09 23:39:51.184,0.000000,20,0.0,13,7.947986e+06,54.940145,2,-2.154382,18,-28.113643,0.000000,5,396541,312,NaN,NaN,NaN,NaN,POINT (-156.36956 71.24096)
2,2023-01-09 23:39:51.187,0.000000,20,0.0,10,7.948006e+06,57.283934,2,2.246301,18,-25.295689,0.000000,5,396542,312,NaN,NaN,NaN,NaN,POINT (-156.36963 71.24113)
3,2023-01-09 23:39:51.697,0.000000,20,0.0,10,7.951628e+06,291.900461,2,11.447052,18,2.851770,0.000000,5,396723,312,NaN,NaN,NaN,NaN,POINT (-156.38271 71.27333)
4,2023-01-13 23:31:33.621,0.505195,50,0.0,58,7.943253e+06,13.065198,0,-0.006597,18,10.703961,0.066336,2,396305,373,NaN,NaN,NaN,NaN,POINT (-156.95342 71.20634)


In [14]:
gf['mosaic.value'].describe()

count    537.000000
mean       0.807525
std        1.944718
min       -1.418108
25%       -0.926480
50%       -0.400521
75%        2.791387
max        6.903812
Name: mosaic.value, dtype: float64

### Difference Sliderule Elevation - ArcticDEM elevation

In [31]:
gfa = gpd.read_file(aoi_url)
m = gfa.explore(style_kwds=dict(fill=False, color='magenta')) 

points = gf.sample(1000).reset_index()
points['time'] = points.time.dt.strftime('%Y-%m-%d')
points.explore(column='h_mean', m=m)

In [32]:
m = gfa.explore(style_kwds=dict(fill=False, color='magenta')) 
points = gf.dropna().reset_index()
points['time'] = points.time.dt.strftime('%Y-%m-%d')
points.explore(column='mosaic.value',m=m)